# Spark SQL
## Spark SQL简介
### Shark
Shark即Hive on Spark，为了实现与Hive兼容，Shark在HiveQL方面重用了Hive中HiveQL的解析、逻辑执行计划翻译、执行计划优化等逻辑，可以近似认为仅将物理执行计划从MapReduce作业替换为Spark作业，通过Hive的HiveQL解析，把HiveQL翻译成Spark上的RDD操作

Shark的设计导致了两个问题
- 执行计划优化完全依赖于Hive，不方便添加新的优化策略
- Spark是线程级并行，而MapReduce是进程级并行，因此，Shark在兼容Hive的实现上存在线程安全问题


### Spark SQL设计
Spark SQL在Hive兼容层面仅依赖HiveQL解析、Hive元数据。从HQL被解析成抽象语法树（AST）起，就全部由Spark SQL接管了。Spark SQL执行计划生成和优化都由Catalyst负责

- Spark SQL增加了DataFrame（即带有Schema信息的RDD），使用户可以在Spark SQL中执行SQL语句，数据即可以来自RDD，也可以是Hive、HDFS、Cassandra等外部数据源，还可以是JSON格式数据


### 为什么推出Spark SQL
- Spark SQL提供了DataFrame API，可以对内部和外部各种数据源执行各种关系型操作
- 其次，可以支持大数据中的大量数据源和数据分析算法。Spark SQL可以融合：传统关系数据库的结构化管理能力和机器学习算法的数据处理能力


## DataFrame概述
- DataFrame的推出，让Spark具备了处理大规模结构化数据的能力，不仅比原有的RDD转化方式更加简单易用，而且有更高的计算性能
- Spark能够轻松实现从MySQL到DataFrame的转化，并且支持SQL查询


**DataFrame与RDD的区别**
- RDD是分布式的Java对象的集合，但是，对象内部结构对RDD而言却不可知
- DataFrame是一种以RDD为基础的分布式数据集，提供了详细的结构信息

## DataFrame的创建
- 从Spark2.0以上版本开始，Spark使用全新的SparkSession接口
- SparkSeesion支持从不同的数据源加载数据，并把数据转换成DataFrame，并且支持把DataFrame转换成SQLContext自身中的表，然后使用SQL语句来操作数据。

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf = SparkConf()).getOrCreate()

在创建DataFrame时，可以使用spark.read操作，从不同类型的文件中加载数据创建DF，例如：
- spark.read.text('people.txt'):读取文本文件people.txt创建DF
- spark.read.json('people.txt'):读取people.json文件创建DF；在本地读取时要给出正确的路径
- spark.read.parquet('people.parquet'):读取people.parquet文件创建DF

或者

- spark.read.format("text").load("people.txt")：读取文本文件people.json创建DataFrame；
- spark.read.format("json").load("people.json")：读取JSON文件people.json创建DataFrame；
- spark.read.format("parquet").load("people.parquet")：读取Parquet文件people.parquet创建DataFrame。

In [2]:
df = spark.read.json('file:///opt/code/SparkProgramming-PySpark/resources/people.json')
# df = spark.read.text('file:///opt/code/SparkProgramming-PySpark/resources/people.txt')
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## DataFrame的保存
可以使用spark.write操作，把一个DataFrame保存成不同格式的文件，例如，把一个名称为df的DataFrame保存到不同格式文件中
- df.write.text('people.txt')
- df.write.json('people.json')
- df.write.parquet('people.parquet')

或者

- df.write.format("text").save("people.txt")
- df.write.format("json").save("people.json")
- df.write.format ("parquet").save("people.parquet")

In [3]:
peopleDF = spark.read.format('json').load('file:///opt/code/SparkProgramming-PySpark/resources/people.json')
peopleDF.select('name', 'age').write.format('json').\
    save('file:///opt/code/SparkProgramming-PySpark/resources/newpeople.json')

# peopleDF.select('name', 'age').write.format('text').\
#     save('file:///opt/code/SparkProgramming-PySpark/resources/newpeople.txt')

## DataFrame的常用操作

In [4]:
df = spark.read.json('resources/people.json')

In [5]:
#printSchema
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [7]:
#select()
df.select(df['name'], df['age']).show()

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+



In [9]:
#filter()
df.filter(df['age']>20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [10]:
#groupBy
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [11]:
#sort()
df.sort(df['age'].desc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [12]:
df.sort(df['age'].desc(), df['name'].desc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



## 从RDD转换得到DataFrame
### 利用反射机制推断RDD模式

In [13]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row, SparkSession

people = spark.sparkContext.textFile('file:///opt/software/spark/examples/src/main/resources/people.txt')\
    .map(lambda line: line.split(',')).map(lambda p: Row(name=p[0], age=int(p[1])))
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView('people')
personsDF = spark.sql('select name,age from people where age > 20')
personsRDD = personsDF.rdd.map(lambda p: 'Name: '+ p.name + ',' + 'Age: '+ str(p.age))
personsRDD.collect()

['Name: Michael,Age: 29', 'Name: Andy,Age: 30']

#### 使用编程方式定义RDD模式
无法提前获取数据结构时，就需要采用编程的方式定义RDD模式
比如，需要通过编程方式把people.txt加载进来生成DataFrame，并完成SQL查询

In [15]:
from pyspark.sql.types import *
from pyspark.sql import Row

schemaString = 'name age'
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split(' ')]
schema = StructType(fields)
lines = spark.sparkContext.textFile('file:///opt/software/spark/examples/src/main/resources/people.txt')
parts = lines.map(lambda x: x.split(','))
people = parts.map(lambda p: Row(p[0], p[1].strip()))
schemaPeople = spark.createDataFrame(people, schema)

In [16]:
schemaPeople.createOrReplaceTempView('people')
results = spark.sql('select name, age from people')
results.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



## 使用Spark SQL读写数据库
### 准备工作
- 启动MySQL
- 创建数据库和表
```mysql
create database spark;
use spark;
create table student (id int(4), name char(20), gender char(4), age int(4));
insert into student values(1, 'Xueqian', 'F', 23);
insert into student values(2, 'Weiliang', 'M', 24);
select * from student;
```

- 将JDBC驱动拷贝到$SPARK_HOME/jars

### 读取MySQL数据库中的数据

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('readMysql')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()
jdbcDF = spark.read.format('jdbc')\
    .option('driver', 'com.mysql.jdbc.Driver')\
    .option('url', 'jdbc:mysql://localhost:3306/spark')\
    .option('dbtable', 'student')\
    .option('user', 'root')\
    .option('password', 'Passwd1!').load()
jdbcDF.show()

+---+--------+------+---+
| id|    name|gender|age|
+---+--------+------+---+
|  1| Xueqian|     F| 23|
|  2|Weiliang|     M| 25|
+---+--------+------+---+



### 向MySQL数据库写入数据

In [6]:
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf= SparkConf()).getOrCreate()

#下面设置模式信息
schema = StructType([StructField('id', IntegerType(), True), \
                    StructField('name', StringType(), True),\
                    StructField('gender', StringType(), True),\
                    StructField('age', IntegerType(), True)])

#设置两条数据，表示学生信息
studentRDD = spark.sparkContext.parallelize(['3 Rongcheng M 26', '4 Guanhua M 27'])\
                        .map(lambda x:x.split(' '))

#创建Row对象，每个Row对象都是rowRDD中的一行
rowRDD = studentRDD.map(lambda p: Row(int(p[0].strip()), p[1].strip(), p[2].strip(), int(p[3].strip())))

#建立Row对象和模式之间的对应关系，也就是把数据和模式对应起来
studentDF = spark.createDataFrame(rowRDD, schema)

#写入数据库
prop = {}
prop['user'] = 'root'
prop['password'] = 'Passwd1!'
prop['driver'] = 'com.mysql.jdbc.Driver'
studentDF.write.jdbc('jdbc:mysql://localhost:3306/spark', 'student', 'append', prop)